# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [94]:
import psycopg2
import pandas as pd
from scipy import stats
import seaborn as sns
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import scipy
import numpy as np
pd.set_option('display.max_columns', None)  #set pandas to display all columns
import sklearn
import sklearn.linear_model

%matplotlib inline

conn = psycopg2.connect(database='mid_term_project',user='lhl_student',password='lhl_student', host='mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com', port='5432')

print('Connection opened successfully')

Connection opened successfully


In [6]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [7]:
column_names = ['fl_date','mkt_unique_carrier','op_unique_carrier','mkt_carrier_fl_num','tail_num','origin','origin_city_name','dest','dest_city_name','crs_dep_time','crs_arr_time','crs_elapsed_time','distance','arr_delay']
data = postgresql_to_dataframe(conn, 'SELECT fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay FROM flights WHERE cancelled = 0 and diverted = 0', column_names)


In [8]:
data = data.dropna()

In [9]:
data = data.sample(5000000)

In [12]:
data

,fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay
6860809,2019-06-03,WN,WN,510,N8639B,BDL,"Hartford, CT",DEN,"Denver, CO",1640,1905,265.0,1671.0,36.0
4051918,2019-07-25,AA,AA,12,N923NN,DFW,"Dallas/Fort Worth, TX",JAX,"Jacksonville, FL",1509,1830,141.0,918.0,32.0
8364893,2019-08-11,WN,WN,4462,N8552Z,MDW,"Chicago, IL",PVD,"Providence, RI",2215,125,130.0,842.0,-17.0
4749850,2018-01-29,AA,MQ,3652,N508AE,ORD,"Chicago, IL",RAP,"Rapid City, SD",1007,1135,148.0,779.0,5.0
14953380,2018-02-18,AS,AS,9,N462AS,MCO,"Orlando, FL",SEA,"Seattle, WA",700,1040,400.0,2554.0,-22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941847,2018-07-07,DL,DL,2610,N954AT,ATW,"Appleton, WI",MSP,"Minneapolis, MN",639,749,70.0,236.0,-16.0
602412,2018-04-20,WN,WN,736,N964WN,DAL,"Dallas, TX",SLC,"Salt Lake City, UT",920,1105,165.0,999.0,234.0
4551899,2018-01-18,AA,AA,1786,N972UY,PHL,"Philadelphia, PA",MCO,"Orlando, FL",1815,2102,167.0,861.0,-25.0
9371080,2019-02-27,UA,OO,5993,N956SW,SFO,"San Francisco, CA",MFR,"Medford, OR",1605,1739,94.0,329.0,114.0


In [11]:
data.to_csv('training_data.csv',index=False)

In [52]:
data = pd.read_csv('training_data.csv')

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [59]:
data['week'] = pd.to_datetime(data['fl_date']).dt.week

In [60]:
data['fl_date'] = pd.to_datetime(data['fl_date'])

In [61]:
data

,fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,week
0,2019-06-03,WN,WN,510,N8639B,BDL,"Hartford, CT",DEN,"Denver, CO",1640,1905,265.0,1671.0,36.0,23
1,2019-07-25,AA,AA,12,N923NN,DFW,"Dallas/Fort Worth, TX",JAX,"Jacksonville, FL",1509,1830,141.0,918.0,32.0,30
2,2019-08-11,WN,WN,4462,N8552Z,MDW,"Chicago, IL",PVD,"Providence, RI",2215,125,130.0,842.0,-17.0,32
3,2018-01-29,AA,MQ,3652,N508AE,ORD,"Chicago, IL",RAP,"Rapid City, SD",1007,1135,148.0,779.0,5.0,5
4,2018-02-18,AS,AS,9,N462AS,MCO,"Orlando, FL",SEA,"Seattle, WA",700,1040,400.0,2554.0,-22.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2018-07-07,DL,DL,2610,N954AT,ATW,"Appleton, WI",MSP,"Minneapolis, MN",639,749,70.0,236.0,-16.0,27
4999996,2018-04-20,WN,WN,736,N964WN,DAL,"Dallas, TX",SLC,"Salt Lake City, UT",920,1105,165.0,999.0,234.0,16
4999997,2018-01-18,AA,AA,1786,N972UY,PHL,"Philadelphia, PA",MCO,"Orlando, FL",1815,2102,167.0,861.0,-25.0,3
4999998,2019-02-27,UA,OO,5993,N956SW,SFO,"San Francisco, CA",MFR,"Medford, OR",1605,1739,94.0,329.0,114.0,9


#### Origin Mean Delay Per Week

In [62]:
column_names = ['fl_date','arr_delay','origin']
origin_delay = postgresql_to_dataframe(conn, 'SELECT fl_date,arr_delay,origin FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
origin_delay['week'] = pd.to_datetime(origin_delay['fl_date']).dt.week
origin_mean_delay = origin_delay.groupby(['origin', origin_delay['week']]).mean().reset_index().rename(columns={'arr_delay':'mean_weekly_origin_delay'})
origin_mean_delay.to_csv('origin_weekly_mean_delay.csv',index=False)
origin_mean_delay

,origin,week,mean_weekly_origin_delay
0,ABE,1,12.616915
1,ABE,2,8.386740
2,ABE,3,9.852941
3,ABE,4,3.483146
4,ABE,5,13.156069
...,...,...,...
19077,YUM,48,-7.142857
19078,YUM,49,-6.371429
19079,YUM,50,11.333333
19080,YUM,51,18.750000


#### Destination Mean Delay Per Week

In [63]:
column_names = ['fl_date','arr_delay','dest']
dest_delay = postgresql_to_dataframe(conn, 'SELECT fl_date,arr_delay,dest FROM flights WHERE cancelled = 0 and diverted = 0', column_names)
dest_delay['week'] = pd.to_datetime(dest_delay['fl_date']).dt.week
dest_mean_delay = dest_delay.groupby(['dest', dest_delay['week']]).mean().reset_index().rename(columns={'arr_delay':'mean_weekly_dest_delay'})
dest_mean_delay.to_csv('dest_weekly_mean_delay.csv',index=False)
dest_mean_delay

,dest,week,mean_weekly_dest_delay
0,ABE,1,4.920000
1,ABE,2,8.855556
2,ABE,3,8.362573
3,ABE,4,4.302198
4,ABE,5,3.350877
...,...,...,...
19078,YUM,48,1.095238
19079,YUM,49,1.300000
19080,YUM,50,3.876712
19081,YUM,51,11.736842


In [64]:
data = data.merge(dest_mean_delay, how='left', left_on=['dest','week'], right_on=['dest','week'])
data = data.merge(origin_mean_delay, how='left', left_on=['origin','week'], right_on=['origin','week'])

In [65]:
data

,fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay
0,2019-06-03,WN,WN,510,N8639B,BDL,"Hartford, CT",DEN,"Denver, CO",1640,1905,265.0,1671.0,36.0,23,9.397645,0.388465
1,2019-07-25,AA,AA,12,N923NN,DFW,"Dallas/Fort Worth, TX",JAX,"Jacksonville, FL",1509,1830,141.0,918.0,32.0,30,11.691860,8.926219
2,2019-08-11,WN,WN,4462,N8552Z,MDW,"Chicago, IL",PVD,"Providence, RI",2215,125,130.0,842.0,-17.0,32,17.007712,9.245760
3,2018-01-29,AA,MQ,3652,N508AE,ORD,"Chicago, IL",RAP,"Rapid City, SD",1007,1135,148.0,779.0,5.0,5,11.728571,16.738499
4,2018-02-18,AS,AS,9,N462AS,MCO,"Orlando, FL",SEA,"Seattle, WA",700,1040,400.0,2554.0,-22.0,7,7.526256,4.523517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2018-07-07,DL,DL,2610,N954AT,ATW,"Appleton, WI",MSP,"Minneapolis, MN",639,749,70.0,236.0,-16.0,27,-2.412042,1.546667
4999996,2018-04-20,WN,WN,736,N964WN,DAL,"Dallas, TX",SLC,"Salt Lake City, UT",920,1105,165.0,999.0,234.0,16,-0.052058,9.078740
4999997,2018-01-18,AA,AA,1786,N972UY,PHL,"Philadelphia, PA",MCO,"Orlando, FL",1815,2102,167.0,861.0,-25.0,3,1.167089,-2.974661
4999998,2019-02-27,UA,OO,5993,N956SW,SFO,"San Francisco, CA",MFR,"Medford, OR",1605,1739,94.0,329.0,114.0,9,7.980989,11.755168


#### Airport Traffic

In [68]:
dest_airport_traffic = dest_delay[['week','dest']].groupby(['dest','week']).size().reset_index(name='arriving_flights_per_week')

In [72]:
dest_airport_traffic.to_csv('dest_weekly_airport_traffic.csv',index=False)

In [69]:
origin_airport_traffic = origin_delay[['week','origin']].groupby(['origin','week']).size().reset_index(name='leaving_flights_per_week')

In [73]:
origin_airport_traffic.to_csv('origin_weekly_airport_traffic.csv',index=False)

In [70]:
data = data.merge(dest_airport_traffic, how='left', left_on=['dest','week'], right_on=['dest','week'])
data = data.merge(origin_airport_traffic, how='left', left_on=['origin','week'], right_on=['origin','week'])

In [71]:
data

,fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week
0,2019-06-03,WN,WN,510,N8639B,BDL,"Hartford, CT",DEN,"Denver, CO",1640,1905,265.0,1671.0,36.0,23,9.397645,0.388465,10869,1181
1,2019-07-25,AA,AA,12,N923NN,DFW,"Dallas/Fort Worth, TX",JAX,"Jacksonville, FL",1509,1830,141.0,918.0,32.0,30,11.691860,8.926219,1378,12118
2,2019-08-11,WN,WN,4462,N8552Z,MDW,"Chicago, IL",PVD,"Providence, RI",2215,125,130.0,842.0,-17.0,32,17.007712,9.245760,778,3361
3,2018-01-29,AA,MQ,3652,N508AE,ORD,"Chicago, IL",RAP,"Rapid City, SD",1007,1135,148.0,779.0,5.0,5,11.728571,16.738499,140,11087
4,2018-02-18,AS,AS,9,N462AS,MCO,"Orlando, FL",SEA,"Seattle, WA",700,1040,400.0,2554.0,-22.0,7,7.526256,4.523517,6170,5380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2018-07-07,DL,DL,2610,N954AT,ATW,"Appleton, WI",MSP,"Minneapolis, MN",639,749,70.0,236.0,-16.0,27,-2.412042,1.546667,5732,225
4999996,2018-04-20,WN,WN,736,N964WN,DAL,"Dallas, TX",SLC,"Salt Lake City, UT",920,1105,165.0,999.0,234.0,16,-0.052058,9.078740,4495,2667
4999997,2018-01-18,AA,AA,1786,N972UY,PHL,"Philadelphia, PA",MCO,"Orlando, FL",1815,2102,167.0,861.0,-25.0,3,1.167089,-2.974661,5135,5092
4999998,2019-02-27,UA,OO,5993,N956SW,SFO,"San Francisco, CA",MFR,"Medford, OR",1605,1739,94.0,329.0,114.0,9,7.980989,11.755168,263,6192


#### Airport Encoding

In [76]:
#mainline = 0, regional = 1, ferry = 2
data['flight_type'] = np.select (
    [ 
     (data['mkt_carrier_fl_num'] .between(1, 2949, inclusive=True)),
     (data['mkt_carrier_fl_num'].between(2950, 7999, inclusive=True)),
     (data['mkt_carrier_fl_num'].between(8000, 9400, inclusive=True)),
    ], 
    [
     'mainline',
     'regional',
     'ferry'
    ]
                                    )
data = pd.concat([data,pd.get_dummies(data['flight_type'], prefix = 'flight_type')], axis = 1)
print(data.shape)
data.head()

(5000000, 23)


,fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type,flight_type_ferry,flight_type_mainline,flight_type_regional
0,2019-06-03,WN,WN,510,N8639B,BDL,"Hartford, CT",DEN,"Denver, CO",1640,1905,265.0,1671.0,36.0,23,9.397645,0.388465,10869,1181,mainline,0,1,0
1,2019-07-25,AA,AA,12,N923NN,DFW,"Dallas/Fort Worth, TX",JAX,"Jacksonville, FL",1509,1830,141.0,918.0,32.0,30,11.691860,8.926219,1378,12118,mainline,0,1,0
2,2019-08-11,WN,WN,4462,N8552Z,MDW,"Chicago, IL",PVD,"Providence, RI",2215,125,130.0,842.0,-17.0,32,17.007712,9.245760,778,3361,regional,0,0,1
3,2018-01-29,AA,MQ,3652,N508AE,ORD,"Chicago, IL",RAP,"Rapid City, SD",1007,1135,148.0,779.0,5.0,5,11.728571,16.738499,140,11087,regional,0,0,1
4,2018-02-18,AS,AS,9,N462AS,MCO,"Orlando, FL",SEA,"Seattle, WA",700,1040,400.0,2554.0,-22.0,7,7.526256,4.523517,6170,5380,mainline,0,1,0


#### Flight Haul Lengths

In [77]:
#short = 0, medium = 1, long = 2
data['flight_length'] = np.select (
    [ 
     (data['distance'] < 800),
     (data['distance'].between(800, 2200, inclusive=False)),
     (data['distance'] >=2200)
    ], 
    [
     'short',
     'medium',
     'long'
    ]
                                    )
data = pd.concat([data,pd.get_dummies(data['flight_length'], prefix = 'flight_haul')], axis = 1)
print(data.shape)
data.head()

(5000000, 27)


,fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_length,flight_haul_long,flight_haul_medium,flight_haul_short
0,2019-06-03,WN,WN,510,N8639B,BDL,"Hartford, CT",DEN,"Denver, CO",1640,1905,265.0,1671.0,36.0,23,9.397645,0.388465,10869,1181,mainline,0,1,0,medium,0,1,0
1,2019-07-25,AA,AA,12,N923NN,DFW,"Dallas/Fort Worth, TX",JAX,"Jacksonville, FL",1509,1830,141.0,918.0,32.0,30,11.691860,8.926219,1378,12118,mainline,0,1,0,medium,0,1,0
2,2019-08-11,WN,WN,4462,N8552Z,MDW,"Chicago, IL",PVD,"Providence, RI",2215,125,130.0,842.0,-17.0,32,17.007712,9.245760,778,3361,regional,0,0,1,medium,0,1,0
3,2018-01-29,AA,MQ,3652,N508AE,ORD,"Chicago, IL",RAP,"Rapid City, SD",1007,1135,148.0,779.0,5.0,5,11.728571,16.738499,140,11087,regional,0,0,1,short,0,0,1
4,2018-02-18,AS,AS,9,N462AS,MCO,"Orlando, FL",SEA,"Seattle, WA",700,1040,400.0,2554.0,-22.0,7,7.526256,4.523517,6170,5380,mainline,0,1,0,long,1,0,0


#### Adding Hourly Arrival and Departure Times

In [78]:
data['crs_dep_hour'] = (data['crs_dep_time'] // 100)
data['crs_arr_hour'] = (data['crs_arr_time'] // 100)

#### Breaking fl_date into year, month day

In [80]:
data['year'] = pd.DatetimeIndex(data['fl_date']).year
data['month'] = pd.DatetimeIndex(data['fl_date']).month
data['day'] = pd.DatetimeIndex(data['fl_date']).day

In [81]:
data

,fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_length,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,year,month,day
0,2019-06-03,WN,WN,510,N8639B,BDL,"Hartford, CT",DEN,"Denver, CO",1640,1905,265.0,1671.0,36.0,23,9.397645,0.388465,10869,1181,mainline,0,1,0,medium,0,1,0,16,19,2019,6,3
1,2019-07-25,AA,AA,12,N923NN,DFW,"Dallas/Fort Worth, TX",JAX,"Jacksonville, FL",1509,1830,141.0,918.0,32.0,30,11.691860,8.926219,1378,12118,mainline,0,1,0,medium,0,1,0,15,18,2019,7,25
2,2019-08-11,WN,WN,4462,N8552Z,MDW,"Chicago, IL",PVD,"Providence, RI",2215,125,130.0,842.0,-17.0,32,17.007712,9.245760,778,3361,regional,0,0,1,medium,0,1,0,22,1,2019,8,11
3,2018-01-29,AA,MQ,3652,N508AE,ORD,"Chicago, IL",RAP,"Rapid City, SD",1007,1135,148.0,779.0,5.0,5,11.728571,16.738499,140,11087,regional,0,0,1,short,0,0,1,10,11,2018,1,29
4,2018-02-18,AS,AS,9,N462AS,MCO,"Orlando, FL",SEA,"Seattle, WA",700,1040,400.0,2554.0,-22.0,7,7.526256,4.523517,6170,5380,mainline,0,1,0,long,1,0,0,7,10,2018,2,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2018-07-07,DL,DL,2610,N954AT,ATW,"Appleton, WI",MSP,"Minneapolis, MN",639,749,70.0,236.0,-16.0,27,-2.412042,1.546667,5732,225,mainline,0,1,0,short,0,0,1,6,7,2018,7,7
4999996,2018-04-20,WN,WN,736,N964WN,DAL,"Dallas, TX",SLC,"Salt Lake City, UT",920,1105,165.0,999.0,234.0,16,-0.052058,9.078740,4495,2667,mainline,0,1,0,medium,0,1,0,9,11,2018,4,20
4999997,2018-01-18,AA,AA,1786,N972UY,PHL,"Philadelphia, PA",MCO,"Orlando, FL",1815,2102,167.0,861.0,-25.0,3,1.167089,-2.974661,5135,5092,mainline,0,1,0,medium,0,1,0,18,21,2018,1,18
4999998,2019-02-27,UA,OO,5993,N956SW,SFO,"San Francisco, CA",MFR,"Medford, OR",1605,1739,94.0,329.0,114.0,9,7.980989,11.755168,263,6192,regional,0,0,1,short,0,0,1,16,17,2019,2,27


In [82]:
data = pd.concat([data,pd.get_dummies(data['mkt_unique_carrier'], prefix = 'carrier')], axis = 1)
print(data.shape)
data.head()

(5000000, 43)


,fl_date,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_length,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,year,month,day,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN
0,2019-06-03,WN,WN,510,N8639B,BDL,"Hartford, CT",DEN,"Denver, CO",1640,1905,265.0,1671.0,36.0,23,9.397645,0.388465,10869,1181,mainline,0,1,0,medium,0,1,0,16,19,2019,6,3,0,0,0,0,0,0,0,0,0,0,1
1,2019-07-25,AA,AA,12,N923NN,DFW,"Dallas/Fort Worth, TX",JAX,"Jacksonville, FL",1509,1830,141.0,918.0,32.0,30,11.691860,8.926219,1378,12118,mainline,0,1,0,medium,0,1,0,15,18,2019,7,25,1,0,0,0,0,0,0,0,0,0,0
2,2019-08-11,WN,WN,4462,N8552Z,MDW,"Chicago, IL",PVD,"Providence, RI",2215,125,130.0,842.0,-17.0,32,17.007712,9.245760,778,3361,regional,0,0,1,medium,0,1,0,22,1,2019,8,11,0,0,0,0,0,0,0,0,0,0,1
3,2018-01-29,AA,MQ,3652,N508AE,ORD,"Chicago, IL",RAP,"Rapid City, SD",1007,1135,148.0,779.0,5.0,5,11.728571,16.738499,140,11087,regional,0,0,1,short,0,0,1,10,11,2018,1,29,1,0,0,0,0,0,0,0,0,0,0
4,2018-02-18,AS,AS,9,N462AS,MCO,"Orlando, FL",SEA,"Seattle, WA",700,1040,400.0,2554.0,-22.0,7,7.526256,4.523517,6170,5380,mainline,0,1,0,long,1,0,0,7,10,2018,2,18,0,1,0,0,0,0,0,0,0,0,0


In [87]:
feat_engineered_training_data = data.drop(['fl_date','mkt_unique_carrier','op_unique_carrier','tail_num','origin','origin_city_name','dest','dest_city_name','crs_dep_time','crs_arr_time','distance','flight_type','flight_length','year'],axis=1)
feat_engineered_training_data

,mkt_carrier_fl_num,crs_elapsed_time,arr_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,month,day,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN
0,510,265.0,36.0,23,9.397645,0.388465,10869,1181,0,1,0,0,1,0,16,19,6,3,0,0,0,0,0,0,0,0,0,0,1
1,12,141.0,32.0,30,11.691860,8.926219,1378,12118,0,1,0,0,1,0,15,18,7,25,1,0,0,0,0,0,0,0,0,0,0
2,4462,130.0,-17.0,32,17.007712,9.245760,778,3361,0,0,1,0,1,0,22,1,8,11,0,0,0,0,0,0,0,0,0,0,1
3,3652,148.0,5.0,5,11.728571,16.738499,140,11087,0,0,1,0,0,1,10,11,1,29,1,0,0,0,0,0,0,0,0,0,0
4,9,400.0,-22.0,7,7.526256,4.523517,6170,5380,0,1,0,1,0,0,7,10,2,18,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2610,70.0,-16.0,27,-2.412042,1.546667,5732,225,0,1,0,0,0,1,6,7,7,7,0,0,0,1,0,0,0,0,0,0,0
4999996,736,165.0,234.0,16,-0.052058,9.078740,4495,2667,0,1,0,0,1,0,9,11,4,20,0,0,0,0,0,0,0,0,0,0,1
4999997,1786,167.0,-25.0,3,1.167089,-2.974661,5135,5092,0,1,0,0,1,0,18,21,1,18,1,0,0,0,0,0,0,0,0,0,0
4999998,5993,94.0,114.0,9,7.980989,11.755168,263,6192,0,0,1,0,0,1,16,17,2,27,0,0,0,0,0,0,0,0,1,0,0


In [88]:
feat_engineered_training_data.to_csv('Feat_engineered_training_data.csv', index=False)

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [89]:
training_df = pd.read_csv('Feat_engineered_training_data.csv')

In [90]:
training_df

,mkt_carrier_fl_num,crs_elapsed_time,arr_delay,week,mean_weekly_dest_delay,mean_weekly_origin_delay,arriving_flights_per_week,leaving_flights_per_week,flight_type_ferry,flight_type_mainline,flight_type_regional,flight_haul_long,flight_haul_medium,flight_haul_short,crs_dep_hour,crs_arr_hour,month,day,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_G4,carrier_HA,carrier_NK,carrier_UA,carrier_VX,carrier_WN
0,510,265.0,36.0,23,9.397645,0.388465,10869,1181,0,1,0,0,1,0,16,19,6,3,0,0,0,0,0,0,0,0,0,0,1
1,12,141.0,32.0,30,11.691860,8.926219,1378,12118,0,1,0,0,1,0,15,18,7,25,1,0,0,0,0,0,0,0,0,0,0
2,4462,130.0,-17.0,32,17.007712,9.245760,778,3361,0,0,1,0,1,0,22,1,8,11,0,0,0,0,0,0,0,0,0,0,1
3,3652,148.0,5.0,5,11.728571,16.738499,140,11087,0,0,1,0,0,1,10,11,1,29,1,0,0,0,0,0,0,0,0,0,0
4,9,400.0,-22.0,7,7.526256,4.523517,6170,5380,0,1,0,1,0,0,7,10,2,18,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2610,70.0,-16.0,27,-2.412042,1.546667,5732,225,0,1,0,0,0,1,6,7,7,7,0,0,0,1,0,0,0,0,0,0,0
4999996,736,165.0,234.0,16,-0.052058,9.078740,4495,2667,0,1,0,0,1,0,9,11,4,20,0,0,0,0,0,0,0,0,0,0,1
4999997,1786,167.0,-25.0,3,1.167089,-2.974661,5135,5092,0,1,0,0,1,0,18,21,1,18,1,0,0,0,0,0,0,0,0,0,0
4999998,5993,94.0,114.0,9,7.980989,11.755168,263,6192,0,0,1,0,0,1,16,17,2,27,0,0,0,0,0,0,0,0,1,0,0


In [92]:
X = training_df.drop(['arr_delay'], axis=1)
y = training_df['arr_delay']

In [95]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, test_size=0.2)

In [96]:
lr = sklearn.linear_model.LinearRegression()

In [97]:
lr.fit(X_train,y_train)

LinearRegression()

In [98]:
y_pred = lr.predict(X_test)

In [99]:
R2 = sklearn.metrics.r2_score(y_test,y_pred)

In [100]:
R2

0.03639972180938178

In [101]:
lr.coef_

array([ 6.43813034e-04, -1.23312431e-02,  6.57265456e-03,  6.89992246e-01,
        6.80776135e-01,  2.28379601e-04,  4.99184883e-06,  4.97047593e+01,
       -2.45710584e+01, -2.51337010e+01,  2.05757590e-01,  1.92570461e-02,
       -2.25014636e-01,  6.89935863e-01,  3.56961479e-01, -1.15643330e-02,
       -1.99338863e-02, -1.84711515e+00, -1.37764376e+00,  4.45522797e+00,
       -3.19476184e+00,  5.23148529e+00,  1.41661520e+00,  3.30458446e+00,
       -1.08120162e+00, -5.22195307e-01, -4.75163817e+00, -1.63335708e+00])

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [102]:
import xgboost as xgb

In [104]:
X = training_df.drop(['arr_delay'], axis=1)
y = training_df['arr_delay']

In [105]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [106]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [124]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 30, alpha = 5, n_estimators = 30)

In [125]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=30,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=30, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=5, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [126]:
y_pred = xg_reg.predict(X_test)

In [127]:
R2 = sklearn.metrics.r2_score(y_test,y_pred)

In [128]:
R2

-0.017612722523525415

In [129]:
from sklearn.metrics import mean_squared_error

In [130]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 50.720383


In [131]:
import pickle

In [132]:
filename = 'xgeeboooost.pickle'
pickle.dump(xg_reg, open(filename, 'wb'))

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.